In [ ]:
# Basic imports
import nbimporter
import logging
import json
import random
import time
import asyncio
import threading
from datetime import date, datetime

# Library imports
import pandas as pd
import numpy as np
import pyarrow as pa

# pyEX is an easy-to-use IEX API interface built for Python
import pyEX

# The main course
import perspective

logging.basicConfig(format="%(asctime)s %(message)s", level=logging.INFO)

# Streaming Data Sources

Inside `datasources.ipynb`, there are a few streaming datasources that will feed live data to Perspective. 

Each datasource runs on its own subprocess and subthread in order to not block the main Jupyter thread from running, so cells can still be added and evaluated as normal. In the background, the datasource will fetch data, clean it (if necessary), and update the Perspective tables—which will display the new results in each widget in the notebook.

In [24]:
from datasources import IEXIntervalDataSource, IEXStaticDataSource
from schemas import last_quote_schema, holdings_schema, charts_schema

# Create a pyEX client with the token - this is just an example sandbox token.
token = "Tpk_ecc89ddf30a611e9958142010a80043c"
client = pyEX.Client(api_token=token, version="sandbox")

### Our Portfolio

For this demonstration, let's set up a fictional portfolio of stocks—it's one of the most natural use cases for streaming data, and it provides a way for us to join static and streaming data together intuitively. In a more comprehensive example, our holdings of individual stocks will probably change over time, but we'll keep it fixed for now.

In [ ]:
symbols = ["AAPL", "MSFT", "AMZN", "TSLA", "SPY", "SNAP", "ZM"]
holdings = {symbol: random.randint(5, 10) for symbol in symbols}

To save our portfolio, we're going to use two Perspective tables:

- `holdings_table`, which is indexed on `symbol` and will always return the latest value of our portfolio based on the prices for each component.
- `holdings_total_table`, which is not indexed, and will hold a history of prices and values for each symbol, allowing us to see the value of our portfolio over time.

Using `on_update`, we link the two tables together; whenever `holdings_table` updates from the datasource, it will pass the updated rows to `holdings_total_table`.

In [ ]:
# Create the table from schema
holdings_table = perspective.Table(holdings_schema, index="symbol")

# Update it with the symbols and quantities of each stock
holdings_table.update({
    "symbol": symbols,
    "quantity": [holdings[symbol] for symbol in symbols]
})

# Create the unindexed total table
holdings_total_table = perspective.Table(holdings_schema)
holdings_view = holdings_table.view()

def update_total(port, delta):
    """When the indexed table updates with the latest price, update the unindexed table with the rows that changed."""
    holdings_total_table.update(delta)

holdings_view.on_update(update_total, mode="row")

Using our indexed `holdings_table`, we can create a new `PerspectiveWidget` to view the Table in Jupyterlab. Using `PerspectiveWidget`'s configuration options, we can set up the view to be exactly what we want—to show the latest price and value for our portfolio.

### Computed Columns

We now have the price and the quantity of each holding, but how would we calculate the value? We can do so entirely within Perspective, using the Computed Expressions UI on the widget. Using a simple, minimal expression language with syntax highlighting and type checking, we can calculate the value of each holding by multiplying the price by the quantity across each row.

In [ ]:
holdings_widget = perspective.PerspectiveWidget(
    holdings_table,
    aggregates={
        "value": "sum not null",
        "price": "last"
    },
    row_pivots=["symbol"],
    columns=["price", "quantity", "value"],
    sort=[["value", "desc"]],
    computed_columns=[{
        "column": "value", 
        "computed_function_name": "*",
        "inputs": ["quantity", "price"]
    }]
)
holdings_widget

And do the same for our `holdings_total_table`—here, we see a line chart of the portfolio value as new prices tick in, split by each symbol so we can see how the portfolio's total value is divided amongst each component.

In [ ]:
holdings_total_widget = perspective.PerspectiveWidget(
    holdings_total_table,
    plugin="y_line",
    row_pivots=["time"],
    column_pivots=["symbol"],
    aggregates={
        "quantity": "last",
        "price": "last"
    },
    columns=["value"],
    computed_columns=[{
        "column": "value", 
        "computed_function_name": "*",
        "inputs": ["quantity", "price"]
    }]
)
holdings_total_widget

### Joining streaming and static data sources

Now that we can see our portfolio values, let's feed the real-time price of each symbol into the portfolio tables. To do this, we'll create another `Table` with
`last_quote_schema`, which conforms to the output from IEX's [Last](https://iexcloud.io/docs/api/#last) endpoint. This API provides "a near real time, intraday API that provides IEX last sale price, size and time," and is perfect for calculating the value of our portfolio quickly.

We'll use `on_update` again—when `quotes_table` updates with a new quote, feed it into `holdings_table`. This will update the latest price of the symbol in our holdings, which will then (with the `on_update` callback we created earlier) feed the `holdings_total_table`. Thus, data flows from the streaming datasource.

In [ ]:
quotes_table = perspective.Table(last_quote_schema)
quotes_view = quotes_table.view()

def update_holdings(port, delta):
    holdings_table.update(delta)
    
quotes_view.on_update(update_holdings, mode="row")

We can create another widget - here we want to see the last (latest) price for each symbol, sorted by price descending:

In [ ]:
quotes_widget = perspective.PerspectiveWidget(
    quotes_table,
    row_pivots=["symbol"],
    columns=["price"],
    aggregates={"price": "last"},
    sort=[["price", "desc"]])

quotes_total_widget = perspective.PerspectiveWidget(
    quotes_table,
    row_pivots=["symbol"],
    columns=["price"],
    aggregates={"price": "last"},
    sort=[["price", "desc"]])

display(quotes_widget, quotes_total_widget)

### Starting our datasource

Now we can start the streaming datasource by providing it with a table and a function that returns data. Because `quote` is not implemented with Server-Sent Events (SSE), we want to manually poll the server every second—which works just as well for our purposes.

In [ ]:
# Clean the quotes to have the right format for sandbox data, which comes with randomly generated `time`s
def clean_quote(tick):
    for t in tick:
        t["time"] = datetime.now()
    return tick

quotes = IEXIntervalDataSource(table=quotes_table, iex_source=client.last, data_cleaner=clean_quote, symbols=symbols)

# Start the subprocess and thread
quotes.start()

### Backtesting

Seeing live data is great, but what if we want to see how our portfolio has performed over time? By joining together Perspective tables, we can use IEX's [Historical Prices](https://iexcloud.io/docs/api/#historical-prices) endpoint to fetch historical data for backtesting. Then, we'll fetch our portfolio holdings from `holdings_table`, and join our holdings with the historical price data.

In [ ]:
# charts_schema conforms to the output of Historical Prices, with `quantity` added so we can easily join it with `holdings_table`.
charts_table = perspective.Table(charts_schema)

In our data cleaner function, we transform the chart data from the API to the correct format, and add our holdings for each symbol.

We don't need to poll the API for historical prices - getting everything in one shot is enough. Here, we'll get data for the last year (controlled by the `range_` kwarg).

In [ ]:
def clean_charts(tick):
    out = []
    for k, v in tick.items():
        chart = v["chart"]
        for c in chart:
            c["symbol"] = k
            c["quantity"] = holdings[k]
            out.append(c)
    return out

# range_: 1d, 1m, 1y, etc.
charts = IEXStaticDataSource(charts_table, iex_source=client.batch, data_cleaner=clean_charts, symbols=symbols, fields="chart", range_="5y")
charts.start()

Create some more widgets - we can pass in the config as a dictionary, which allows us to switch quickly between widget configurations.

- `ohlc_config` shows the Open, High, Low, and Close prices for `SPY`, which tracks the S&P 500 index. Changing the filter (or removing it entirely) will show exactly the symbols you want to see.
- `value_config` computes the `value` column, which uses the quantity and each day's closing price to calculate the value of each symbol at the end of the day. This allows us to see the value of our portfolio split by symbol in the past year.

In [ ]:
ohlc_config = {
    "plugin": "d3_ohlc",
    "row_pivots": ["date"],
    "columns": ["open", "close", "high", "low"],
    "aggregates": {"quantity": "last"},
    "filters": [["symbol", "==", "SPY"]]
}

value_config = {
    "plugin": "y_line",
    "row_pivots": ["date"],
    "column_pivots": ["symbol"],
    "columns": ["value"],
    "aggregates": {"quantity": "last"},
    "computed_columns": [{
        "column": "value", 
        "computed_function_name": "*",
        "inputs": ["quantity", "close"]
    }]
}

charts_widget = perspective.PerspectiveWidget(
    charts_table,
    **ohlc_config
)
charts_widget

### Saving to Apache Arrow

Now we have our portfolio tables set up, how would we archive the value of our portfolio over time? One way is to write to [Apache Arrow](https://arrow.apache.org/), a storage format for columnar data that is lightning-fast to read and write.

We can set up a separate thread that calls the view's `to_arrow()` method every 60 seconds, and dump the current state of the holdings total table every minute. Using the `minute_bucket` computed column, we can bucket the price updates into minutes instead of keeping data from each second. With row and column pivots on `minute_bucket` and `symbol`, we can show the value of the portfolio split across each symbol for every minute. This allows us to persist data beyond the life-cycle of the notebook kernel.

In [ ]:
save_holdings_view = holdings_total_table.view(
    aggregates={
        "value": "last",
        "quantity": "last"
    },
    column_pivots=["symbol"],
    row_pivots=["minute_bucket"],
    columns=["symbol", "quantity", "value", "time"],
    computed_columns=[
        {
            "column": "value", 
            "computed_function_name": "*",
            "inputs": ["quantity", "price"]
        },
        {
            "column": "minute_bucket",
            "computed_function_name": "minute_bucket",
            "inputs": ["time"]
        }
    ]
)

stop_save = False

async def _save():
    while True:
        if stop_save:
            return

        name = "portfolio_value_{0:%Y_%m_%d}.arrow".format(datetime.today())

        if save_holdings_view.num_rows() > 0:
            with open(name, "wb") as value_arrow:
                value_arrow.write(save_holdings_view.to_arrow())
            logging.info("Saved %d rows to %s", holdings_total_table.size(), name)
        await asyncio.sleep(60)

def save_to_arrow():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    task = loop.create_task(_save())
    loop.run_until_complete(task)
    
save_thread = threading.Thread(target=save_to_arrow)
save_thread.start()

In [ ]:
with open("portfolio_value_{0:%Y_%m_%d}.arrow".format(datetime.today()), "rb") as arr:
    w = perspective.PerspectiveWidget(arr.read(), sort=[["time", "desc"]])
    display(w)